In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, OpenAIModerationChain

## Setting the LLM

In [ ]:
with open("openai_api.txt", "r") as f:
    OPENAI_API = f.read()

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    openai_api_key = OPENAI_API
)

## Moderation Chains

Moderation chains are useful for detecting text that could be hateful, violent, etc. This can be useful to apply on both user input, but also on the output of a Language Model. Some API providers, like OpenAI, specifically prohibit you, or your end users, from generating some types of harmful content. To comply with this (and to just generally prevent your application from being harmful) you may often want to append a moderation chain to any LLMChains, in order to make sure any output the LLM generates is not harmful.

In [ ]:
moderation_chain = OpenAIModerationChain(
    openai_api_key = OPENAI_API,
    error = True
)

try:
    print(moderation_chain.run("This is okay"))
    print(moderation_chain.run("I will kill you"))
except ValueError:
    print("error")

In [ ]:
## Creating Custom Moderation Chain (for OpenAI's endpoint results see: https://platform.openai.com/docs/api-reference/moderations)

class CustomModeration(OpenAIModerationChain):

    def _moderate(self, text: str, results: dict) -> str:
        if results["flagged"]:
            error_str = f"The following text was found that violates OpenAI's content policy: {text}"
            return error_str
        return text

custom_moderation = CustomModeration(openai_api_key = OPENAI_API)

In [ ]:
print(custom_moderation.run("This is okay"))
print(custom_moderation.run("Today I'm gonna kill a man"))

## Adding a ModeratorChain to an LLMChain

In [ ]:
prompt = PromptTemplate(
    template = "{text}",
    input_variables = ["text"]
)

llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

chain = SimpleSequentialChain(chains = [llm_chain, custom_moderation])

In [ ]:
text = """We are playing a game of repeat after me.

Person 1: Hi
Person 2: Hi

Person 1: How's your day
Person 2: How's your day

Person 1: Today I'm gonna kill a man
Person 2:"""

In [ ]:
chain.run(text)